In [ ]:
import psycopg2
import os
from sys import exit

# Configuración de la base de datos (usa variables de entorno o valores por defecto)
DB_CONFIG = {
    "host": os.getenv("DB_HOST", "localhost"),  # "db" en Docker, "localhost" local
    "user": os.getenv("DB_USER", "admin"),
    "password": os.getenv("DB_PASSWORD", "secret"),
    "database": os.getenv("DB_NAME", "tictactoe")
}

def create_db_connection():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except psycopg2.Error as e:
        print(f"\nError al conectar a la base de datos: {e}\n")
        return None

def init_db():
    conn = create_db_connection()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS games (
                    id SERIAL PRIMARY KEY,
                    winner VARCHAR(10),
                    board_state VARCHAR(20)
                );
            """)
            conn.commit()
            cursor.close()
            print("\n¡Tabla 'games' creada o ya existente!\n")
        except psycopg2.Error as e:
            print(f"Error al crear tablas: {e}")
        finally:
            conn.close()

def save_game(winner, board):
    conn = create_db_connection()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute(
                "INSERT INTO games (winner, board_state) VALUES (%s, %s)",
                (winner, "".join(board))
            )
            conn.commit()
            cursor.close()
            print("\n¡Partida guardada en la base de datos!\n")
        except psycopg2.Error as e:
            print(f"Error al guardar partida: {e}")
        finally:
            conn.close()

def print_board(board):
    print("\n")
    for i in range(0, 9, 3):
        print(f"{board[i]} | {board[i+1]} | {board[i+2]}")
        if i < 6:
            print("-----")

def check_winner(board):
    win_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Filas
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columnas
        [0, 4, 8], [2, 4, 6]              # Diagonales
    ]
    for condition in win_conditions:
        if board[condition[0]] == board[condition[1]] == board[condition[2]] != " ":
            return board[condition[0]]
    return None

def main():
    init_db()  # Intenta inicializar la DB (no crítico si falla)
    board = [" " for _ in range(9)]  # Tablero vacío
    current_player = "X"

    print("¡Bienvenido al Tic Tac Toe!")
    print_board(["1", "2", "3", "4", "5", "6", "7", "8", "9"])  # Muestra referencia de celdas

    for _ in range(9):
        try:
            move = int(input(f"Jugador {current_player}, elige una casilla (1-9): ")) - 1
            if move < 0 or move > 8 or board[move] != " ":
                print("Casilla inválida. Intenta de nuevo.")
                continue
        except ValueError:
            print("Entrada no válida, introduce un número del 1 al 9.")
            continue

        board[move] = current_player
        print_board(board)

        winner = check_winner(board)
        if winner:
            print(f"¡El jugador {winner} ha ganado!")
            save_game(winner, board)
            exit()

        current_player = "O" if current_player == "X" else "X"

    print("¡Empate!")
    save_game("Draw", board)

if __name__ == "__main__":
    main()


Error al conectar a la base de datos: connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "admin"


¡Bienvenido al Tic Tac Toe!


1 | 2 | 3
-----
4 | 5 | 6
-----
7 | 8 | 9
Entrada no válida, introduce un número del 1 al 9.
